In [10]:
import pandas as pd
import os
# import sys to get more detailed Python exception info
import sys
# import the connect library for psycopg2
import psycopg2
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd
from io import StringIO
import numpy as np
from timeit import default_timer as timer
import postgressql
import mlflow
import tempfile
from sklearn import preprocessing

In [11]:
os.environ['MLFLOW_TRACKING_URI'] = "http://0.0.0.0:8000"
os.environ['MLFLOW_TRACKING_USERNAME'] = "Pablo"

In [12]:
client = mlflow.tracking.MlflowClient()

In [13]:
mlflow.set_experiment("bankchurners_feature_Engineering")

<Experiment: artifact_location='/Users/salv91/Desktop/mlflow_artifactRoot/mlruns/2', experiment_id='2', lifecycle_stage='active', name='bankchurners_feature_Engineering', tags={}>

In [14]:
def Feature_engineering(location): 

    #load the data
    df = pd.read_csv(f"{location}",na_values = ["n.a.","?","NA","n/a", "na", "--","nan"],index_col=False) 
    
    labelencoder = preprocessing.LabelEncoder()
    
    df['Gender'] = labelencoder.fit_transform(df['Gender'])
    
    oneHotCols=["Card_Category","Marital_Status"]
    
    df=pd.get_dummies(df, columns=oneHotCols)
    
    replaceStruct = {
                    "Attrition_Flag": {"Existing Customer":0,"Attrited Customer": 1},
                    "Education_Level": {"Doctorate": 5,"Post-Graduate": 4,"Graduate": 3,"College": 2,"High School": 1,"Unknown": 0,"Uneducated": -1 },
                    "Income_Category": {"$120K +": 4, "$80K - $120K": 3, "$60K - $80K": 2, "$40K - $60K": 1, "Unknown" :0, "Less than $40K": -1}
                        }

    df=df.replace(replaceStruct)

    return df 

In [15]:
with mlflow.start_run() as run:
    
    dataFrame = Feature_engineering(location="/Users/salv91/Desktop/mlflow_artifactRoot/mlruns/1/dd03e24808ee404296c929ee7c13a63b/artifacts/Raw_Data/Raw_Data_ek43an8g.csv")
    
    temp = tempfile.NamedTemporaryFile(prefix="Feature_Data_", suffix=".csv")
    temp_name = temp.name
    
    try:
        dataFrame.to_csv(temp_name, index=False)
        mlflow.log_artifact(temp_name, "Feature_Data")
    finally:
        temp.close() # Delete the temp file

    #print info run
    run_id = run.info.run_id
    mlflow.set_tag("run_id", run_id)
    print(run.info)
    mlflow.end_run()

<RunInfo: artifact_uri='/Users/salv91/Desktop/mlflow_artifactRoot/mlruns/2/f458ae6da86f4003bfdd93625c7d466f/artifacts', end_time=None, experiment_id='2', lifecycle_stage='active', run_id='f458ae6da86f4003bfdd93625c7d466f', run_uuid='f458ae6da86f4003bfdd93625c7d466f', start_time=1654009686296, status='RUNNING', user_id='salv91'>
